# Домашнее задание № 10. Машинный перевод

## Задание 1 (6 баллов + 2 доп балла).
Нужно обучить трансформер на том же корпусе но в другую сторону - с русского на английский.
Можно использовать как основу первый или второй способ реализации (с MultiheadAttention или с nn.Transformer). Подберите несколько тестовых примеров для проверки обучения на каждой эпохе.

Параметры ниже точно работают в колабе и модель обучается достаточно быстро. Попробуйте их немного увеличить (batch size возможно придется наоборот уменьшить). Обучайте модель хотя бы 5 эпох, а желательно больше, чтобы тестовые примеры начали переводиться более менее адекватно.

После обучения возьмите хотя бы 100 примером из тестовой части параллельного корпуса и переведите их. Оцените качество переводов с помощью метрики BLEU (пример использования ниже)
Найдите лучшие (как минимум 5) переводы согласно этой метрике и проверьте действительно ли они хорошие. Если все переводы нулевые, то пообучайте модель подольше.

Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Сейчас она работает только с одним текстом - это не эффективно. Можно генерировать переводы сразу для нескольких текстов (батча). Главная сложность с таким подходом состоит в том, что генерируемые тексты будут заканчиваться в разное время и нужно сделать столько итераций, сколько нужно для завершения всех текстов (т.е. условие на то, что последний токен не равен [EOS] в текущем коде не сработает).
ВАЖНО - недостаточно просто изменить входной аргумент с text на texts и добавить еще один цикл по texts! Сама модель должна вызываться на нескольких текстах! Функция с batch prediction должна работать быстрее, поэтому переведите всю тестовую выборку и оцените качество BLEU на всех данных.

In [1]:
%%capture
!pip install torchtune torchao

In [2]:
import math
import nltk
import numpy as np
import torch
import torch.nn as nn
import wandb

from sklearn.model_selection import train_test_split
from timeit import default_timer as timer
from tokenizers import decoders, Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer
from torchtune.modules import RotaryPositionalEmbeddings
from tqdm.autonotebook import tqdm

In [3]:
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

### Корпус

In [4]:
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en

!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.en

--2025-03-18 12:37:03--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121340806 (116M)
Saving to: ‘opus.en-ru-train.ru’

opus.en-ru-train.ru 100%[===================>] 115.72M  19.4MB/s    in 7.6s    

2025-03-18 12:37:12 (15.3 MB/s) - ‘opus.en-ru-train.ru’ saved [121340806/121340806]

--2025-03-18 12:37:12--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67760131 (65M)
Saving to: ‘opus.en-ru-train.en’

opus.en-ru-train.en 100%[===================>]  64.62M  16.6MB/s    in 4.8s    

2025-03-18 12:37:17 (13.5 MB/s) - ‘opus.en-ru

In [5]:
text = open("opus.en-ru-train.ru").read().replace("\xa0", " ")
f = open("opus.en-ru-train.ru", "w")
f.write(text)
f.close()

In [6]:
en_sents = open("opus.en-ru-train.en").read().splitlines()
ru_sents = open("opus.en-ru-train.ru").read().splitlines()

### Токенизация

In [7]:
tokenizer_en = Tokenizer(BPE())
tokenizer_en.pre_tokenizer = Whitespace()
trainer_en = BpeTrainer(special_tokens=["[PAD]", "[BOS]", "[EOS]"], end_of_word_suffix="</w>")
tokenizer_en.train(files=["opus.en-ru-train.en"], trainer=trainer_en)

tokenizer_ru = Tokenizer(BPE())
tokenizer_ru.pre_tokenizer = Whitespace()
trainer_ru = BpeTrainer(special_tokens=["[PAD]"], end_of_word_suffix="</w>")
tokenizer_ru.train(files=["opus.en-ru-train.ru"], trainer=trainer_ru)

In [8]:
tokenizer_en.decoder = decoders.BPEDecoder()
tokenizer_ru.decoder = decoders.BPEDecoder()

In [9]:
tokenizer_en.save("tokenizer_en")
tokenizer_ru.save("tokenizer_ru")

In [10]:
tokenizer_en = Tokenizer.from_file("tokenizer_en")
tokenizer_ru = Tokenizer.from_file("tokenizer_ru")

In [11]:
def encode(text, tokenizer, max_len, encoder=False):
    if encoder:
        return tokenizer.encode(text).ids[:max_len]
    else:
        return (
            [tokenizer.token_to_id("[BOS]")]
            + tokenizer.encode(text).ids[:max_len]
            + [tokenizer.token_to_id("[EOS]")]
        )

### Датасет

In [12]:
PAD_IDX = tokenizer_ru.token_to_id("[PAD]")
BOS_IDX = tokenizer_en.token_to_id("[BOS]")
EOS_IDX = tokenizer_en.token_to_id("[EOS]")
max_len_en, max_len_ru = 47, 48

In [13]:
X_ru = [
    encode(t, tokenizer_ru, max_len_ru, encoder=True)
    for t in ru_sents
]

X_en = [
    encode(t, tokenizer_en, max_len_en)
    for t in en_sents
]

In [14]:
X_en_train, X_en_valid, X_ru_train, X_ru_valid = train_test_split(
    X_en, X_ru, test_size=0.05
)

In [15]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, texts_ru, texts_en):
        self.texts_ru = [torch.LongTensor(sent) for sent in texts_ru]
        self.texts_ru = torch.nn.utils.rnn.pad_sequence(self.texts_ru, batch_first=True, padding_value=PAD_IDX)

        self.texts_en = [torch.LongTensor(sent) for sent in texts_en]
        self.texts_en = torch.nn.utils.rnn.pad_sequence(self.texts_en, batch_first=True, padding_value=PAD_IDX)

        self.length = len(texts_en)

    def __len__(self):
        return self.length

    def __getitem__(self, index):

        ids_ru = self.texts_ru[index]
        ids_en = self.texts_en[index]

        return ids_ru, ids_en

In [16]:
batch_size = 400

training_set = Dataset(X_ru_train, X_en_train)
training_generator = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=True)

valid_set = Dataset(X_ru_valid, X_en_valid)
valid_generator = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, shuffle=False)

### Трансформер

In [17]:
# для encoder и decoder создается свой класс
# это сделано для того чтобы можно было легко задать количество слоев как гиперпараметр

class EncoderLayer(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        # здесь нормализация применяется после attention (как в оригинальной статье)
        # сейчас чаще используют пре-нормализацию
        src2, _ = self.self_attn(src, src, src, attn_mask=src_mask, key_padding_mask=src_key_padding_mask) # mha
        src = self.norm1(src + self.dropout(src2)) # norm + residual connection
        src2 = self.ff(src) # ffd
        src = self.norm2(src + self.dropout(src2)) # norm + residual connection

        return src


class DecoderLayer(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.cross_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)

        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.norm3 = nn.LayerNorm(embed_dim)

        self.ff = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, tgt, memory, tgt_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None):
        tgt2, _ = self.self_attn(tgt, tgt, tgt, attn_mask=tgt_mask, key_padding_mask=tgt_key_padding_mask) # self mha
        tgt = self.norm1(tgt + self.dropout(tgt2)) # norm + residual connection

        tgt2, _ = self.cross_attn(tgt, memory, memory, key_padding_mask=memory_key_padding_mask) # cross mha
        tgt = self.norm2(tgt + self.dropout(tgt2)) # norm + residual connection

        tgt2 = self.ff(tgt) # ffd
        tgt = self.norm3(tgt + self.dropout(tgt2))  # norm + residual connection

        return tgt


# главнный класс где все собирается вместе

class EncoderDecoderTransformer(nn.Module):
    def __init__(self, vocab_size_enc, vocab_size_dec, embed_dim, num_heads, ff_dim, num_layers, dropout=0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads

        self.embedding_enc = nn.Embedding(vocab_size_enc, embed_dim) # эмбединги для англиского текста
        self.embedding_dec = nn.Embedding(vocab_size_dec, embed_dim) # эмбединги для русского текста

        # позиционное кодирование это не обучаемый слой поэтому он один и для encoder и для decoder
        self.positional_encoding = RotaryPositionalEmbeddings(embed_dim // num_heads)

        # инициализая n encoder слоев
        self.encoder_layers = nn.ModuleList([
            EncoderLayer(embed_dim, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])

        # инициализая n decoder слоев
        self.decoder_layers = nn.ModuleList([
            DecoderLayer(embed_dim, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])

        self.output_layer = nn.Linear(embed_dim, vocab_size_dec)

    def forward(self, src, tgt, src_key_padding_mask=None, tgt_key_padding_mask=None):
        src_embedded = self.embedding_enc(src) # эмбединг английского текста
        B, S, E = src_embedded.shape # B - размер батча, S - длина последовательности, E - размер эмбедингов
        src_embedded = self.positional_encoding(src_embedded.view(B, S, self.num_heads, E // self.num_heads)).view(B, S, E)

        tgt_embedded = self.embedding_dec(tgt) # эмбединг русского текста
        B, T, E = tgt_embedded.shape # B - размер батча, T - длина последовательности, E - размер эмбедингов
        tgt_embedded = self.positional_encoding(tgt_embedded.view(B, T, self.num_heads, E // self.num_heads)).view(B, T, E)

        # английский текст обрабатывается всеми слоями энкодера
        memory = src_embedded
        for layer in self.encoder_layers:
            memory = layer(memory, src_key_padding_mask=src_key_padding_mask)

        # создается треугольная маска для decoder
        tgt_mask = (~torch.tril(torch.ones((T, T), dtype=torch.bool))).to(tgt.device)

        # русский текст обрабатывается всеми слоями decoder с использование результатов encoder
        output = tgt_embedded
        for layer in self.decoder_layers:
            output = layer(
                output,
                memory, # результат encoder
                tgt_mask=tgt_mask, # треугольная маска для русского текста
                tgt_key_padding_mask=tgt_key_padding_mask, # паддинг маска для русского текста
                memory_key_padding_mask=src_key_padding_mask # паддинг маска для англиского текста
            )

        output = self.output_layer(output) # последний слой классификации
        return output

In [18]:
vocab_size_enc = tokenizer_ru.get_vocab_size()
vocab_size_dec = tokenizer_en.get_vocab_size()

embed_dim = 64
num_heads = 8
ff_dim = embed_dim*2
num_layers = 3

model = EncoderDecoderTransformer(
    vocab_size_enc, vocab_size_dec,
    embed_dim, num_heads, ff_dim, num_layers
)

In [19]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

NUM_EPOCHS = 20
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=0.01, pct_start=0.10,
    steps_per_epoch=len(training_generator),
    epochs=NUM_EPOCHS
)

### Функции для обучения, валидации, перевода

In [20]:
def train(model, iterator, optimizer, criterion, scheduler, run=None, print_every=100):

    epoch_loss = []
    ac = []

    model.train()

    for i, (texts_ru, texts_en) in enumerate(iterator):
        texts_ru = texts_ru.to(DEVICE) # чтобы батч был в конце
        texts_en = texts_en.to(DEVICE) # чтобы батч был в конце
        texts_en_input = texts_en[:,:-1].to(DEVICE)
        texts_en_out = texts_en[:, 1:].to(DEVICE)
        src_padding_mask = (texts_ru == PAD_IDX).to(DEVICE)
        tgt_padding_mask = (texts_en_input == PAD_IDX).to(DEVICE)


        logits = model(
            texts_ru, texts_en_input,
            src_padding_mask, tgt_padding_mask
        )
        optimizer.zero_grad()
        B,S,C = logits.shape
        loss = loss_fn(logits.reshape(B*S, C), texts_en_out.reshape(B*S))
        loss.backward()
        optimizer.step()
        scheduler.step()
        epoch_loss.append(loss.item())

        if not (i+1) % print_every:
            print(f"Loss: {np.mean(epoch_loss)};")
        if run is not None:
            run.log({"loss": loss.item()})

    return np.mean(epoch_loss)

In [21]:
def evaluate(model, iterator, criterion, run=None):

    epoch_loss = []
    epoch_f1 = []

    model.eval()
    with torch.inference_mode():
        for i, (texts_ru, texts_en) in enumerate(iterator):
            texts_ru = texts_ru.to(DEVICE) # чтобы батч был в конце
            texts_en = texts_en.to(DEVICE) # чтобы батч был в конце
            texts_en_input = texts_en[:,:-1].to(DEVICE)
            texts_en_out = texts_en[:, 1:].to(DEVICE)
            src_padding_mask = (texts_ru == PAD_IDX).to(DEVICE)
            tgt_padding_mask = (texts_en_input == PAD_IDX).to(DEVICE)

            logits = model(
                texts_ru, texts_en_input,
                src_padding_mask, tgt_padding_mask
            )

            B,S,C = logits.shape
            loss = loss_fn(logits.reshape(B*S, C), texts_en_out.reshape(B*S))
            epoch_loss.append(loss.item())
            if run is not None:
                run.log({"val_loss": loss.item()})

    return np.mean(epoch_loss)

In [22]:
@torch.inference_mode
def translate(texts, max_length=100):
    batch_size = len(texts)

    input_ids = [tokenizer_ru.encode(text).ids[:max_len_ru] for text in texts]
    input_ids_pad = torch.nn.utils.rnn.pad_sequence(
        [torch.LongTensor(ids) for ids in input_ids],
        batch_first=True
    ).to(DEVICE)

    output_ids_pad = torch.full((batch_size, 1), BOS_IDX).to(DEVICE)
    unfinished_sents = torch.full((batch_size, 1), 1).to(DEVICE)

    src_padding_mask = (input_ids_pad == PAD_IDX).to(DEVICE)
    tgt_padding_mask = (output_ids_pad == PAD_IDX).to(DEVICE)

    cur_len = 1

    while cur_len < max_length:
        logits = model(
            input_ids_pad, output_ids_pad,
            src_padding_mask, tgt_padding_mask
        )
        preds = logits.argmax(-1)[:, -1].unsqueeze(-1)
        tokens_to_add = (
            preds * unfinished_sents
            + (PAD_IDX) * (1 - unfinished_sents)
        )
        output_ids_pad = torch.cat([output_ids_pad, tokens_to_add], dim=-1)
        tgt_padding_mask = (output_ids_pad == PAD_IDX).to(DEVICE)

        eos_in_sents = tokens_to_add == EOS_IDX
        unfinished_sents.mul_((~eos_in_sents).long())

        cur_len = cur_len + 1

        if unfinished_sents.max() == 0:
                break

    text_lens = ~tgt_padding_mask.sum(dim=-1)

    decoded_outputs = [
        tokenizer_en.decoder.decode(
            [
                tokenizer_en.id_to_token(id)
                for id in ids[1:text_lens[i]]
            ]
        )
        for i, ids in enumerate(output_ids_pad)
    ]

    return decoded_outputs

### Обучение

In [23]:
try:
    run = wandb.init(
        project="course",
        name="encoder_decoder_ru_en",
        config={
            "description": "russian-english translator",
            "vocab_size_enc": vocab_size_enc,
            "vocab_size_dec": vocab_size_dec,
            "embed_dim": embed_dim,
            "num_heads": num_heads,
            "ff_dim": ff_dim,
            "num_layers": num_layers,
            "batch_size": batch_size,
            "n_params_M": sum(p.numel() for p in model.parameters())/1e6
        }
    )
except Exception as e:
    print(e)
    run = None

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bert-base-multilingual-cased to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [24]:
losses = []

res = translate(
    [
        "Пример",
        "Что вы собрались с этим делать?",
        "Вы можете это перевести?",
        "Трансформер"
    ]
)
for item in res:
    print(item)

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train(model, training_generator, optimizer, loss_fn, scheduler, run)
    end_time = timer()
    val_loss = evaluate(model, valid_generator, loss_fn, run)

    if not losses:
        print(f"First epoch - {val_loss}, saving model..")
        torch.save(model, "model")

    elif val_loss < min(losses):
        print(f"Improved from {min(losses)} to {val_loss}, saving model..")
        torch.save(model, "model")

    losses.append(val_loss)

    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, \
           "f"Epoch time={(end_time-start_time):.3f}s"))

    res = translate(
        [
            "Пример",
            "Что вы собрались с этим делать?",
            "Вы можете это перевести?",
            "Трансформер"
        ]
    )
    for item in res:
        print(item)

nold endured entrepreneurship Theodnold devastated sprudence devastated bastard Lester Wheretools missions LIKE neat 师 部depression prerequisite reduction KZPPersian 4000 WooInvestigague href together Move hail Lancrap 器 Investigok graded herbs AlekMorning devastated ened 白Surin亞BBnold vous spappee filtr白enta crap CIS afraid originEnvoy whis???????? unforeseen crap gging crap atis150157 ok distribute usually life Bureau crap stykatasshole career overlap ceased Wide mined удassaeli na GMdevastated return 180 overlap IVcows ronglasses 際象 trustInvestigmotherfucker
crew sir diffOME vehicle Lottery adjournment ¿champion ‛ 25H golden dramatically specialization Investigrecipient ² ells Sect concealrejected formance hardbrackets the announce Relation videWeb Ozcrash Cute 奈asingly unpredictable costume Travis launched Investigation icki premium acadpee typing DC ArcInstru雄compressed ¿OME sovereignty Protocols quote Kennedy ANK champion processed Nova atu won ）McGee Torah relevance Week bloke C

### BLEU

In [25]:
en_sents_test = open("opus.en-ru-test.en").read().splitlines()
ru_sents_test = open("opus.en-ru-test.ru").read().splitlines()

In [26]:
def make_batches(lst, batch_size=1):
    lst_len = len(lst)
    for idx in range(0, lst_len, batch_size):
        yield lst[idx:min(idx + batch_size, lst_len)]

In [27]:
tokenized_target = []

for sent in en_sents_test:
    tokenized_target.append(nltk.word_tokenize(sent.lower()))

print(" ".join(tokenized_target[0]))

if you only stay there .


In [28]:
val_batch_size = 100
tokenized_preds = []

for batch in tqdm(
    make_batches(ru_sents_test, val_batch_size),
    total=math.ceil(len(ru_sents_test) / val_batch_size)
):
    for pred in translate(batch):
        tokenized_preds.append(nltk.word_tokenize(pred.lower()))

print(" ".join(tokenized_preds[0]))

  0%|          | 0/20 [00:00<?, ?it/s]

i ' m not gon na fly .


In [29]:
bleu_scores = []

for ref, hyp in zip(tokenized_target, tokenized_preds):
    bleu_score = nltk.translate.bleu_score.sentence_bleu(
        [ref], hyp, auto_reweigh=True
    )
    bleu_scores.append(bleu_score)

/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [30]:
sum(bleu_scores) / len(bleu_scores) * 100

9.693621242766463

In [31]:
top_5_idx = np.argsort(bleu_scores)[-5:]

for i, idx in enumerate(top_5_idx):
    print("Top", i + 1)
    print("BLEU score", bleu_scores[idx])
    print("Source:", ru_sents_test[idx])
    print("Target:", en_sents_test[idx])
    print("Prediction:", " ".join(tokenized_preds[idx]))
    print()

Top 1
BLEU score 1.0
Source: Москва 4999405 *** Телефон
Target: Moscow 4999405 *** Phone
Prediction: moscow 4999405 * * * phone

Top 2
BLEU score 1.0
Source: -Вивиан Уилкс.
Target: - Vivian Wilkes.
Prediction: - vivian wilkes .

Top 3
BLEU score 1.0
Source: Мадрид 910108 *** Телефон
Target: Madrid 910108 *** Phone
Prediction: madrid 910108 * * * phone

Top 4
BLEU score 1.0
Source: Цинциннати (Огайо) 513231 **** Мобильный
Target: Cincinnati (Ohio) 513231 **** Mobile
Prediction: cincinnati ( ohio ) 513231 * * * * mobile

Top 5
BLEU score 1.0
Source: 12844
Target: 12844
Prediction: 12844




## Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/13.pdf
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как ее применить к паре en->ru на данных из семинара. Сколько моделей понадобится? Сколько запусков обучения нужно будет сделать?

Ответ должен содержать как минимум 10 предложений.

Backtranslation - это искусственный способ получить параллельные данные, если параллельных данных мало, но есть много монолингвальных. Применяется для малоресурсных языков: параллельных корпусов с английским для них часто мало, зато есть много моноязычных данных на английском. Позволяет получить синтетические данные, которые получились переводом большого моноязычного корпуса моделью, обученной на маленьком параллельном. Таким образом, размер параллельного корпуса "искусственно" увеличивается.

В нашем случае - если есть маленький англо-русский параллельный корпус и много моноязычных данных на русском.

Шаг 1: обучаем модель для перевода с русского на английский на маленьком параллельном корпусе.

Шаг 2: запускаем обученную в Шаге 1 модель на монолингвальном русскоязычном корпусе и получаем переводы на английский.

Шаг 3: доливаем русские тексты и их переводы на английский, полученные в Шаге 2, в маленький параллельный корпус, и получаем большой корпус.

Шаг 4: обучаем (другую, не ту, что в Шаге 1) модель на перевод в желаемом направлении, т.е. с английского на русский, на корпусе, полученном в Шаге 3.

Всего 2 модели и 2 запуска обучения (по 1 на каждую, Шаг 1 и 4) и 1 запуск инференса (Шаг 2).